In [1]:
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyOAuth
import pprint

f = open('login.txt', 'r')
lines = f.readlines()
#-1 to ignore newline character
CLIENT_ID = lines[0][:-1]
CLIENT_SECRET = lines[1][:-1]
REDIRECT_URI = lines[2][:-1]
username = lines[3]

scope = 'user-top-read user-library-read'

token = util.prompt_for_user_token(
    username, scope, client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI)

sp = spotipy.Spotify(auth=token)

results = sp.current_user_saved_tracks()
tracks = {}
for idx, item in enumerate(results['items']):
    track = item['track']
    print(track['name'])
    tracks[track['id']] = [track['name'], track['artists'][0]['name']]
    # print(idx, track['artists'][0]['name'], "–", track['name'])

# print(tracks)
print(list(tracks.keys())[0])
print()
analysis = sp.audio_analysis(list(tracks.keys())[0])
pp = pprint.PrettyPrinter(indent=2)
# remove print for brevity
# pp.pprint(analysis)

Rubble To Rubble
Are You in Love?
Stella Brown
West Coast Love
Pheromones
I'm God
Best for You
May I Have This Dance
Waterfall
That's Where I Am
I Can't Help But Feel
Wave of You
The Best Part of Life (Imanbek Remix)
California Dreamin' - Single Version
Blood Type
Better Now (feat. MARO)
2055
SAD GIRLZ LUV MONEY Remix (feat. Kali Uchis and Moliy)
Hollow (feat. Emma Louise)
We Belong - Odesza Remix
3o5K1YmFz9i9KluRGJusd1



In [2]:
import sqlite3
# comment out so I don't accidentally run again and rewrite everything
# con = sqlite3.connect('songs.db')

# cur = con.cursor()
# cur.execute('''drop table if exists tracks''')
# cur.execute(
#     '''create table tracks 
#         (acousticness real not null,
#         artists text not null,
#         danceability real not null,
#         duration_ms real not null,
#         energy real not null,
#         id text not null,
#         instrumentalness real not null,
#         key int not null,
#         liveness real not null,
#         loudness real not null,
#         mode int not null,
#         name text not null,
#         speechiness real not null,
#         tempo real not null,
#         valence real not null,
#         primary key(id))''')
# con.commit()
# con.close()

In [ ]:
import pandas as pd
import os
import json
import time

path = '/Users/nickparis/spotify-rec/spotify_million_playlist_dataset/data/'
files = os.listdir(path)

con = sqlite3.connect('songs.db')
cur = con.cursor()

f = files.index('mpd.slice.946000-946999.json')
print(f)
print(len(files))
# already got first few
files = files[f:]
for file_idx in range(len(files)-f):
    file = files[file_idx]
    print('reading file ' + file)
    data = json.load(open(path+file))
    data = data['playlists']
#     pp.pprint(data)

    if type(data) is not list:
        data = list(data)

    # build object to insert into database
    errs = 0
    for playlist in data:
        # get all track ids from playlist
        #ignore spotify:track:
        ids = [track['track_uri'][14:] for track in playlist['tracks']]
        names = [track['track_name'] for track in playlist['tracks']]
        names = [name.replace("'", "") for name in names]
        artists = [track['artist_name'] for track in playlist['tracks']]
        artists = [artist.replace("'", "") for artist in artists]
        
        try:
            # spotify API supports only 100IDs at once
            while True:
                ids_shortened = ids[0:100]
                names_shortened = names[0:100]
                artists_shortened = artists[0:100]

                features = sp.audio_features(tracks=ids_shortened)
    #             pp = pprint.PrettyPrinter(indent=1)
    #             pp.pprint(features)
                while None in features:
                    idx = features.index(None)
                    features.pop(idx)
                    names_shortened.pop(idx)
                    artists_shortened.pop(idx)


                # populate dataframe for easier insert into db
                df = pd.json_normalize(features)
                df['name'] = names_shortened
                df['artists'] = artists_shortened
                for _, row in df.iterrows():
                    query = \
                    '''INSERT INTO tracks ''' +\
                    '''(acousticness,artists,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,name ,speechiness,tempo,valence)''' +\
                    ''' VALUES ''' +\
                    '''({},{},{},{},{},{},{},{},{},{},{},{},{},{},{});'''\
                    .format(row['acousticness'],'\''+row['artists']+'\'',row['danceability'],row['duration_ms'],row['energy'],'\''+row['id']+'\'',row['instrumentalness'],row['key'],row['liveness'],row['loudness'],row['mode'],'\''+row['name']+'\'' ,row['speechiness'],row['tempo'],row['valence'])
                    try:
                        cur.execute(query)
                    except sqlite3.Error as err:
                        errs += 1
                        if errs == 50000:
                            print('50000 insert errors')
                            print(str(err) + ' ' + row['name'])
                            errs = 0

                con.commit()

                ids = ids[100:]
                names = names[100:]
                artists = artists[100:]

                if len(ids) == 0:
                    break
        except spotipy.client.SpotifyException:
            #refresh token
            token = util.prompt_for_user_token(
    username, scope, client_id=CLIENT_ID, client_secret=CLIENT_SECRET, redirect_uri=REDIRECT_URI)

            sp = spotipy.Spotify(auth=token)
            file_idx -= 1
        except Exception as e:
            print('error')
            print(e)
            print('playlist ' + playlist['name'])
            print('file ' + file)
con.close()
    
    


In [169]:
#identify duplicates

con = sqlite3.connect('songs.db')
query = '''select t1.artists, t1.name, t1.id from tracks t1 inner join ''' + \
    '''tracks t2 on t1.artists=t2.artists and t1.name=t2.name and t1.id<>t2.id order by t1.name'''

dupes = pd.read_sql_query(query, con)
dupes

,artists,name,id
0,Nick Drake,Cello Song,6p6ocbEBSBOswTLG9xSToK
1,Nick Drake,Cello Song,6ab6cQ8CtAoM9nafkEJEZs
2,Eddie Murphy,"""Buckwheat""",4Li1CTGmeRr0utBexNj7nY
3,Eddie Murphy,"""Buckwheat""",00HoZVs6yTHp6hpt0gLc5m
4,Tylman Susato,"""Danserye"" From Het Derde Musijck Boexcken",6551rVcUfxtyIblfeVaWJu
...,...,...,...
81955,EXO,늑대와 미녀 Wolf - Live,4xJx8JihEl6AEeuKc4vfrX
81956,JJ Lin,사랑해요只對你說,3Sv4RD58BLEd0OvPVBp1vg
81957,JJ Lin,사랑해요只對你說,2rvNd2qPkq056yZBbAAfaS
81958,Standing Egg,오래된 노래,5gllIJSLyRouz1bGO5ues4


now we will remove the duplicates

In [177]:
remove = dupes.iloc[::2,:]
cur = con.cursor()
for i in range(remove.shape[0]):
    query = '''delete from tracks where id=\'{}\''''.format(remove.iloc[i]['id'])
    cur.execute(query)
con.commit()
cur.close()

In [178]:
con = sqlite3.connect('songs.db')
cur = con.cursor()
query = '''select * from tracks'''
data = pd.read_sql_query(query, con)
data.memory_usage(deep=True).sum()

385781174

we have ~386MB of song data. Let's save the dataframe in a csv

In [179]:
data.to_csv('data.csv', index=False)

In this file, we make use of the json formatted million playlist song data. For each song, we hit the spotipy api to get audio_features, and throw that information into a sqlite database in case my computer can not handle the memory restrictions. After running about 10 hours, we have 1.1 million songs. This is about half of the 2 million + songs that are listed on the website.

We then dump the song data into a csv which is 154 MB according to what my mac finder says. In the next file we will begin data exploration and analysis.

references
https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge

https://spotipy.readthedocs.io/en/2.19.0/